In [6]:
from datetime import datetime
from elasticsearch import Elasticsearch
from pandas import json_normalize
import pandas as pd
import requests
#client = Elasticsearch("http://localhost:9200")

In [2]:
client = Elasticsearch(hosts="http://localhost:9200",verify_certs=False)
type(client)

elasticsearch.Elasticsearch

In [7]:
corpus= pd.read_json('Data\Archive.json')

In [ ]:
def deleteIndices():
    for i in range(0,200):
        client.indices.delete(index = i)

In [157]:
def createIndices():
    for i in range(0,200):
        client.indices.create(index = i)
        client.indices.put_mapping(index = i,  body={
        "properties": {
            "Title": {
                "type":  "search_as_you_type"
            },
            "Date": {
                "type":  "date"
            },
            "Authors": {
                "type":  "text"
            },
            "Abstract": {
                "type":  "completion"
            },
            "Embeddings": {
                "type":  "float"
            },
            "Month": {
                "type":  "integer"
            },
            "Year": {
                "type":  "long"
            },
            "Link": {
                "type":  "text"
            },
            "TandA": {
                "type":  "keyword"
            },
            "Keywords_new": {
                "type":  "text"
            },
            "DocKeywords": {
                "type":  "text"
            }}})

In [172]:
def index_data(corpus):
  for i, row in (corpus.iterrows()):
      client.index(
      index = row['cluster_id'],
      id = row['DocId'],
      document = {
                  "Title": row['Title'],
                  "Date": row["Date"],
                  "Authors": row["Authors"],
                  "Abstract": row["Abstract"],
                  "Embeddings": row["Embeddings"],
                  "Month": row["Month"],
                  "Year": row["Year"],
                  "Link": row["Link"],
                  "TandA": row["TandA"],
                  "Keywords": row["Keywords_new"],
                  "DocumentKeywords": row["DocKeywords"]
      }
  )

In [7]:
#Get data by cluster ID
def getDataWithClusterId(clusterId):       
       res = client.search(index=clusterId, size= 1000, body={"query": {"match_all": {}}})
       df = json_normalize(res['hits']['hits'])
       df.rename(columns = {'_index':'cluster_id', '_id':'DocId','_source.Title' : 'Title', '_source.Date' : 'Date',
              '_source.Authors': 'Authors', '_source.Abstract': 'Authors', '_source.Embeddings': 'Embeddings',
              '_source.Month' : 'Month', '_source.Year' : 'Year', '_source.Link' :'Link', '_source.TandA':'TandA',
              '_source.Keywords':'Keywords_new', '_source.DocumentKeywords' : 'DocKeywords'}, inplace = True)

       return df

In [10]:
def getDataWithQuery(query):
    res = requests.post('http://localhost:9200/_all/_search?size=200', json={
                                  "query": {"match": {      
                                        "Title": query
                                      }
                                    }}
                              )
    df = json_normalize(res['hits']['hits'])
    df.rename(columns = {'_index':'cluster_id', '_id':'DocId','_source.Title' : 'Title', '_source.Date' : 'Date',
              '_source.Authors': 'Authors', '_source.Abstract': 'Authors', '_source.Embeddings': 'Embeddings',
              '_source.Month' : 'Month', '_source.Year' : 'Year', '_source.Link' :'Link', '_source.TandA':'TandA',
              '_source.Keywords':'Keywords_new', '_source.DocumentKeywords' : 'DocKeywords'}, inplace = True)

    return df


In [9]:
getDataWithClusterId(4).head()

,cluster_id,DocId,_score,Title,Date,Authors,Authors,Embeddings,Month,Year,Link,TandA,Keywords_new,DocKeywords
0,4,56,1.0,A Bayesian Neural Network Approach to identify...,2022-12-30,"[[Gharat, Sarvesh], [Bose, Bhaskar]]","In today&#39;s era, a tremendous amount of dat...","[[-0.25731718540000004, -0.1844584048, 0.09864...",12,2022,http://arxiv.org/pdf/2301.00056.pdf,Bayesian Neural Network Approach identify Star...,"[stars agns observed, agns observed xmm, ident...","[stars agns observed, agns observed xmm, ident..."
1,4,1130,1.0,Statistical model of a superfluid solid,2023-01-03,"[[Yukalov, V.I.], [Yukalova, E.P.]]",A microscopic statistical model of a quantum s...,"[[0.0397386029, -0.09516981990000001, 0.083395...",1,2023,http://arxiv.org/pdf/2301.01130.pdf,Statistical model superfluid solid microscopic...,"[opportunity answer real, overall properties s...","[opportunity answer real, overall properties s..."
2,4,1222,1.0,A Multi-Source Information Learning Framework ...,2023-01-01,"[[Jiang, Lu], [Li, Yuanhan], [Luo, Na], [Wang,...",With the development of technology and sharing...,"[[0.043689671900000004, -0.11773904410000001, ...",1,2023,http://arxiv.org/pdf/2301.01222.pdf,Information Learning Framework Airbnb Price Pr...,"[promising exists deficiencies, pricing proble...","[promising exists deficiencies, pricing proble..."
3,4,1507,1.0,Bifurcation instructed design of multistate ma...,2023-01-04,"[[Yang, Teaya], [Hathcock, David], [Chen, Yuch...",We propose a novel design paradigm for multist...,"[[0.0173554234, 0.0762074366, -0.024930127, -0...",1,2023,http://arxiv.org/pdf/2301.01507.pdf,Bifurcation instructed design multistate machi...,"[multiple transition pathways, making machine ...","[multiple transition pathways, making machine ..."
4,4,1850,1.0,Bayesian Weapon System Reliability Modeling wi...,2023-01-04,"[[Potter, Michael], [Cheng, Benny], [citation_...",We propose to integrate weapon system features...,"[[-0.1427284926, -0.0243010279, -0.0414356627,...",1,2023,http://arxiv.org/pdf/2301.01850.pdf,Bayesian Weapon System Reliability Modeling Ne...,"[models xgboost current, propose integrate wea...","[models xgboost current, propose integrate wea..."
